In [20]:
import pandas as pd
import numpy as np
import os
import random

In [4]:
# ONLY WORKS FOR THIS SPECIFIC DATSET, CHANGE ACCORDINGLY
rootdir = '../geological_similarity'
image_files = []
image_labels = []
for subdir, dirs, files in os.walk(rootdir):
    print(subdir[25:])
    for file in files:
        image_files.append(os.path.join(subdir, file))
        image_labels.append(subdir[25:])


andesite
gneiss
marble
quartzite
rhyolite
schist


In [10]:
label_dict = {}
encoded_labels = []
for l in image_labels:
    if l not in label_dict:
        label_dict[l] = len(label_dict)
    encoded_labels.append(label_dict[l])
label_dict

{'andesite': 0,
 'gneiss': 1,
 'marble': 2,
 'quartzite': 3,
 'rhyolite': 4,
 'schist': 5}

In [13]:
image_locs = pd.DataFrame({'location':image_files, 'label':encoded_labels})
image_locs.head()

,location,label
0,../geological_similarity\andesite\012L6.jpg,0
1,../geological_similarity\andesite\01ITR.jpg,0
2,../geological_similarity\andesite\01LQQ.jpg,0
3,../geological_similarity\andesite\0230P.jpg,0
4,../geological_similarity\andesite\02741.jpg,0


In [17]:
image_locs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29998 entries, 0 to 29997
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   location  29998 non-null  object
 1   label     29998 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 468.8+ KB


In [ ]:
# generating all possible pairs, will take a long time to run (~90 minutes on a fast CPU)
# total_pairs = []
# for i in range(len(image_locs)-1):
#     for j in range(i+1, len(image_locs)):
#         total_pairs.append([image_locs['location'][i], image_locs['location'][j], int(image_locs['label'][i]==image_locs['label'][j])])
# pairs_df = pd.DataFrame(total_pairs, columns=['inputA', 'inputB', 'label'])
# pairs_df.head()

In [ ]:
# matched_df = pairs_df[pairs_df['label']==1].copy()
# mismatched_df = pairs_df[pairs_df['label']==0].copy()

In [ ]:
# shuffling
# matched_df = matched_df.sample(frac=1).reset_index(drop=True)
# mismatched_df = mismatched_df.sample(frac=1).reset_index(drop=True)

In [ ]:
# 50:50 downsampling
# downsampled_df = pd.concat([matched_df, mismatched_df[:len(matched_df)].copy()], ignore_index=True)

In [24]:
# faster approximation of getting a 50:50 downsampling
NUM_ITERS = 1000000  # number of iterations to run - dataset size will be 2x this
total_pairs = []
for k in range(NUM_ITERS):
    i = random.randint(0, len(image_locs)-1)
    # looping to find matching pair
    while True:
        j = random.randint(0, len(image_locs)-1)
        if image_locs['label'][i]==image_locs['label'][j] and i!=j:
            total_pairs.append([image_locs['location'][i], image_locs['location'][j], 1])
            break
    # looping to find mismatched pair
    while True:
        j = random.randint(0, len(image_locs)-1)
        if image_locs['label'][i]!=image_locs['label'][j]:
            total_pairs.append([image_locs['location'][i], image_locs['location'][j], 0])
            break
    if k%100000 == 0:
        print("Done for {} pairs".format(k*2))

Done for 0 pairs
Done for 200000 pairs
Done for 400000 pairs
Done for 600000 pairs
Done for 800000 pairs
Done for 1000000 pairs
Done for 1200000 pairs
Done for 1400000 pairs
Done for 1600000 pairs
Done for 1800000 pairs


In [25]:
downsampled_df = pd.DataFrame(total_pairs, columns=['inputA', 'inputB', 'label'])
downsampled_df.head()

,inputA,inputB,label
0,../geological_similarity\marble\KRW5E.jpg,../geological_similarity\marble\D4LHE.jpg,1
1,../geological_similarity\marble\KRW5E.jpg,../geological_similarity\gneiss\6SR9M.jpg,0
2,../geological_similarity\quartzite\G5W57.jpg,../geological_similarity\quartzite\LSIXH.jpg,1
3,../geological_similarity\quartzite\G5W57.jpg,../geological_similarity\marble\YQCAX.jpg,0
4,../geological_similarity\schist\BCLF6.jpg,../geological_similarity\schist\PY1QD.jpg,1


In [26]:
downsampled_df.describe()

,label
count,2000000.0
mean,0.5
std,0.5
min,0.0
25%,0.0
50%,0.5
75%,1.0
max,1.0


In [27]:
# shuffling and taking train-test split

downsampled_df = downsampled_df.sample(frac=1).reset_index(drop=True)
train_size = int(0.7*len(downsampled_df))
train_df = downsampled_df[:train_size].copy()
test_df = downsampled_df[train_size+1:].copy()
test_df.reset_index(drop=True, inplace=True)

In [32]:
downsampled_df.to_csv('siamese_pairs.csv', index=False)

In [37]:
image_locs.to_csv('geo_image_locations.csv', index=False)